In [1]:
import pandas as pd

In [2]:
ha_costs_df = pd.read_csv('data/mmd_heart_attack_data.csv')

In [3]:
ha_costs_df.head(5)

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
0,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1001,Autauga County,ALABAMA,Urban,All,undefined,40470
1,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1003,Baldwin County,ALABAMA,Rural,All,undefined,36615
2,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1005,Barbour County,ALABAMA,Rural,All,undefined,46509
3,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1007,Bibb County,ALABAMA,Urban,All,undefined,42949
4,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,1009,Blount County,ALABAMA,Urban,All,undefined,50067


In [4]:
ha_costs_df.tail(5)

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
2651,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72097,Mayaguez Municipio,PUERTO RICO,Urban,All,undefined,28128
2652,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72113,Ponce Municipio,PUERTO RICO,Urban,All,undefined,26403
2653,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72125,San German Municipio,PUERTO RICO,Urban,All,undefined,26398
2654,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72127,San Juan Municipio,PUERTO RICO,Urban,All,undefined,36599
2655,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,72139,Trujillo Alto Municipio,PUERTO RICO,Urban,All,undefined,26676


In [6]:
ha_costs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656 entries, 0 to 2655
Data columns (total 17 columns):
year                   2656 non-null int64
geography              2656 non-null object
measure                2656 non-null object
adjustment             2656 non-null object
analysis               2656 non-null object
domain                 2656 non-null object
condition              2656 non-null object
primary_sex            2656 non-null object
primary_age            2656 non-null object
primary_dual           2656 non-null object
fips                   2656 non-null int64
county                 2656 non-null object
state                  2656 non-null object
urban                  2656 non-null object
primary_race           2656 non-null object
primary_denominator    2656 non-null object
analysis_value         2656 non-null int64
dtypes: int64(3), object(14)
memory usage: 352.8+ KB


In [7]:
TN_heart = ha_costs_df.query('state == "TENNESSEE"')

In [8]:
TN_heart

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
2030,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47001,Anderson County,TENNESSEE,Urban,All,undefined,42749
2031,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47003,Bedford County,TENNESSEE,Rural,All,undefined,43661
2032,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47005,Benton County,TENNESSEE,Rural,All,undefined,44423
2033,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47007,Bledsoe County,TENNESSEE,Rural,All,undefined,41240
2034,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47009,Blount County,TENNESSEE,Urban,All,undefined,38968
2035,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47011,Bradley County,TENNESSEE,Urban,All,undefined,40068
2036,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47013,Campbell County,TENNESSEE,Rural,All,undefined,37715
2037,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47015,Cannon County,TENNESSEE,Urban,All,undefined,50604
2038,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47017,Carroll County,TENNESSEE,Rural,All,undefined,44771
2039,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47019,Carter County,TENNESSEE,Urban,All,undefined,45266


In [9]:
TN_heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91 entries, 2030 to 2120
Data columns (total 17 columns):
year                   91 non-null int64
geography              91 non-null object
measure                91 non-null object
adjustment             91 non-null object
analysis               91 non-null object
domain                 91 non-null object
condition              91 non-null object
primary_sex            91 non-null object
primary_age            91 non-null object
primary_dual           91 non-null object
fips                   91 non-null int64
county                 91 non-null object
state                  91 non-null object
urban                  91 non-null object
primary_race           91 non-null object
primary_denominator    91 non-null object
analysis_value         91 non-null int64
dtypes: int64(3), object(14)
memory usage: 12.8+ KB


In [10]:
TN_heart.nunique()

year                    1
geography               1
measure                 1
adjustment              1
analysis                1
domain                  1
condition               1
primary_sex             1
primary_age             1
primary_dual            1
fips                   91
county                 91
state                   1
urban                   2
primary_race            1
primary_denominator     1
analysis_value         91
dtype: int64

In [24]:
TN_heart.query('urban =="Urban"')

,year,geography,measure,adjustment,analysis,domain,condition,primary_sex,primary_age,primary_dual,fips,county,state,urban,primary_race,primary_denominator,analysis_value
2030,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47001,Anderson County,TENNESSEE,Urban,All,undefined,42749
2034,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47009,Blount County,TENNESSEE,Urban,All,undefined,38968
2035,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47011,Bradley County,TENNESSEE,Urban,All,undefined,40068
2037,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47015,Cannon County,TENNESSEE,Urban,All,undefined,50604
2039,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47019,Carter County,TENNESSEE,Urban,All,undefined,45266
2040,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47021,Cheatham County,TENNESSEE,Urban,All,undefined,32279
2041,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47023,Chester County,TENNESSEE,Urban,All,undefined,49264
2048,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47037,Davidson County,TENNESSEE,Urban,All,undefined,50778
2051,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47043,Dickson County,TENNESSEE,Urban,All,undefined,53168
2053,2017,County,Average total cost,Unsmoothed actual,Base measure,Primary chronic conditions,Acute myocardial infarction,All,All,Dual & non-dual,47047,Fayette County,TENNESSEE,Urban,All,undefined,43000


In [25]:
Urb_heart = TN_heart.query('urban =="Urban"')

In [26]:
Urb_heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 2030 to 2120
Data columns (total 17 columns):
year                   38 non-null int64
geography              38 non-null object
measure                38 non-null object
adjustment             38 non-null object
analysis               38 non-null object
domain                 38 non-null object
condition              38 non-null object
primary_sex            38 non-null object
primary_age            38 non-null object
primary_dual           38 non-null object
fips                   38 non-null int64
county                 38 non-null object
state                  38 non-null object
urban                  38 non-null object
primary_race           38 non-null object
primary_denominator    38 non-null object
analysis_value         38 non-null int64
dtypes: int64(3), object(14)
memory usage: 5.3+ KB


In [21]:
Rur_heart = TN_heart.query('urban =="Rural"')

In [23]:
Rur_heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53 entries, 2031 to 2118
Data columns (total 17 columns):
year                   53 non-null int64
geography              53 non-null object
measure                53 non-null object
adjustment             53 non-null object
analysis               53 non-null object
domain                 53 non-null object
condition              53 non-null object
primary_sex            53 non-null object
primary_age            53 non-null object
primary_dual           53 non-null object
fips                   53 non-null int64
county                 53 non-null object
state                  53 non-null object
urban                  53 non-null object
primary_race           53 non-null object
primary_denominator    53 non-null object
analysis_value         53 non-null int64
dtypes: int64(3), object(14)
memory usage: 7.5+ KB


In [27]:
TN_heart.describe()

,year,fips,analysis_value
count,91.0,91.000000,91.000000
mean,2017.0,47093.615385,45762.989011
std,0.0,55.338708,6574.670462
min,2017.0,47001.000000,30831.000000
25%,2017.0,47046.000000,41105.500000
50%,2017.0,47093.000000,45384.000000
75%,2017.0,47142.000000,49698.500000
max,2017.0,47189.000000,62641.000000


In [28]:
Urb_heart.describe()

,year,fips,analysis_value
count,38.0,38.000000,38.000000
mean,2017.0,47103.789474,45748.921053
std,0.0,60.904071,7109.788921
min,2017.0,47001.000000,32279.000000
25%,2017.0,47049.500000,40452.750000
50%,2017.0,47112.000000,45496.500000
75%,2017.0,47160.500000,50734.500000
max,2017.0,47189.000000,61541.000000


In [29]:
Rur_heart.describe()

,year,fips,analysis_value
count,53.0,53.000000,53.000000
mean,2017.0,47086.320755,45773.075472
std,0.0,50.309723,6232.724885
min,2017.0,47003.000000,30831.000000
25%,2017.0,47045.000000,41240.000000
50%,2017.0,47085.000000,45384.000000
75%,2017.0,47123.000000,48998.000000
max,2017.0,47185.000000,62641.000000


In [31]:
cancer_costs_df = pd.read_csv("data/mmd_cancer_data.csv")

In [33]:
cancer_costs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3165 entries, 0 to 3164
Data columns (total 17 columns):
year                   3165 non-null int64
geography              3165 non-null object
measure                3165 non-null object
adjustment             3165 non-null object
analysis               3165 non-null object
domain                 3165 non-null object
condition              3165 non-null object
primary_sex            3165 non-null object
primary_age            3165 non-null object
primary_dual           3165 non-null object
fips                   3165 non-null int64
county                 3165 non-null object
state                  3165 non-null object
urban                  3165 non-null object
primary_race           3165 non-null object
primary_denominator    3165 non-null object
analysis_value         3165 non-null int64
dtypes: int64(3), object(14)
memory usage: 420.4+ KB


In [34]:
TN_cancer = cancer_costs_df.query('state == "TENNESSEE"')

In [35]:
TN_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 2396 to 2490
Data columns (total 17 columns):
year                   95 non-null int64
geography              95 non-null object
measure                95 non-null object
adjustment             95 non-null object
analysis               95 non-null object
domain                 95 non-null object
condition              95 non-null object
primary_sex            95 non-null object
primary_age            95 non-null object
primary_dual           95 non-null object
fips                   95 non-null int64
county                 95 non-null object
state                  95 non-null object
urban                  95 non-null object
primary_race           95 non-null object
primary_denominator    95 non-null object
analysis_value         95 non-null int64
dtypes: int64(3), object(14)
memory usage: 13.4+ KB


In [36]:
TN_cancer.nunique()

year                    1
geography               1
measure                 1
adjustment              1
analysis                1
domain                  1
condition               1
primary_sex             1
primary_age             1
primary_dual            1
fips                   95
county                 95
state                   1
urban                   2
primary_race            1
primary_denominator     1
analysis_value         95
dtype: int64

In [37]:
TN.shape

(91, 17)

In [38]:
TN_cancer.shape

(95, 17)

In [39]:
list(set(TN_cancer.county)-set(TN.county))

['Van Buren County', 'Moore County', 'Hancock County', 'Pickett County']

In [40]:
Urb_cancer = TN_cancer.query('urban == "Urban"')

In [41]:
Urb_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 2396 to 2490
Data columns (total 17 columns):
year                   38 non-null int64
geography              38 non-null object
measure                38 non-null object
adjustment             38 non-null object
analysis               38 non-null object
domain                 38 non-null object
condition              38 non-null object
primary_sex            38 non-null object
primary_age            38 non-null object
primary_dual           38 non-null object
fips                   38 non-null int64
county                 38 non-null object
state                  38 non-null object
urban                  38 non-null object
primary_race           38 non-null object
primary_denominator    38 non-null object
analysis_value         38 non-null int64
dtypes: int64(3), object(14)
memory usage: 5.3+ KB


In [42]:
Rur_cancer = TN_cancer.query('urban == "Rural"')

In [43]:
Rur_cancer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 2397 to 2488
Data columns (total 17 columns):
year                   57 non-null int64
geography              57 non-null object
measure                57 non-null object
adjustment             57 non-null object
analysis               57 non-null object
domain                 57 non-null object
condition              57 non-null object
primary_sex            57 non-null object
primary_age            57 non-null object
primary_dual           57 non-null object
fips                   57 non-null int64
county                 57 non-null object
state                  57 non-null object
urban                  57 non-null object
primary_race           57 non-null object
primary_denominator    57 non-null object
analysis_value         57 non-null int64
dtypes: int64(3), object(14)
memory usage: 8.0+ KB


In [45]:
Urb_cancer.describe()

,year,fips,analysis_value
count,38.0,38.000000,38.000000
mean,2017.0,47103.789474,19638.605263
std,0.0,60.904071,2712.218435
min,2017.0,47001.000000,15454.000000
25%,2017.0,47049.500000,17674.750000
50%,2017.0,47112.000000,19203.500000
75%,2017.0,47160.500000,20556.750000
max,2017.0,47189.000000,27740.000000


In [46]:
Rur_cancer.describe()

,year,fips,analysis_value
count,57.0,57.000000,57.000000
mean,2017.0,47089.140351,19837.421053
std,0.0,50.642810,2255.416100
min,2017.0,47003.000000,15531.000000
25%,2017.0,47049.000000,18328.000000
50%,2017.0,47087.000000,19633.000000
75%,2017.0,47129.000000,21315.000000
max,2017.0,47185.000000,25909.000000


In [47]:
TN_cancer.describe()

,year,fips,analysis_value
count,95.0,95.000000,95.000000
mean,2017.0,47095.000000,19757.894737
std,0.0,55.136195,2436.304533
min,2017.0,47001.000000,15454.000000
25%,2017.0,47048.000000,18103.500000
50%,2017.0,47095.000000,19563.000000
75%,2017.0,47142.000000,21153.000000
max,2017.0,47189.000000,27740.000000


In [48]:
income_df = pd.read_csv("data/irs_county_2016.csv")

In [50]:
income_df.head

<bound method NDFrame.head of        STATEFIPS STATE  COUNTYFIPS       COUNTYNAME  agi_stub      N1   mars1  \
0              1    AL           0          Alabama         1   26450   14680   
1              1    AL           0          Alabama         2  285760  217880   
2              1    AL           0          Alabama         3  529780  259680   
3              1    AL           0          Alabama         4  495750  211100   
4              1    AL           0          Alabama         5  263130   82330   
5              1    AL           0          Alabama         6  167080   29240   
6              1    AL           0          Alabama         7  216280   21250   
7              1    AL           0          Alabama         8   59310    4250   
8              1    AL           1   Autauga County         1     210     110   
9              1    AL           1   Autauga County         2    3210    2570   
10             1    AL           1   Autauga County         3    5410    2830  

In [52]:
TN_income = income_df.query('STATE == "TN"')

In [54]:
TN_income.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 19752 to 20519
Columns: 148 entries, STATEFIPS to A11902
dtypes: int64(146), object(2)
memory usage: 894.0+ KB


In [55]:
TN_income.head()

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
19752,47,TN,0,Tennessee,1,39580,22400,14440,980,25390,...,6760,17669,200,287,0,0,3970,7167,16170,59567
19753,47,TN,0,Tennessee,2,439770,345410,39920,49300,188490,...,109250,62045,0,0,0,0,37380,18477,366150,366510
19754,47,TN,0,Tennessee,3,717430,398720,115040,189290,341750,...,449870,461060,0,0,0,0,63270,70761,635450,1870568
19755,47,TN,0,Tennessee,4,759340,355970,217710,159890,347430,...,629080,1669328,110,100,0,0,104290,176690,646420,1783890
19756,47,TN,0,Tennessee,5,396810,128390,215890,42240,198820,...,382580,2135577,30,29,0,0,87440,220372,302390,849648


In [56]:
TN_income.tail()

,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
20515,47,TN,189,Wilson County,4,14400,7130,4050,2610,6520,...,12340,34654,0,0,0,0,2090,4150,12140,31712
20516,47,TN,189,Wilson County,5,9330,2890,5160,960,4600,...,9010,50591,0,0,0,0,2080,5356,7100,19970
20517,47,TN,189,Wilson County,6,7160,1060,5630,380,3560,...,7110,61964,0,0,0,0,1710,5743,5290,17041
20518,47,TN,189,Wilson County,7,10350,780,9260,240,5330,...,10330,190229,60,13,0,0,3810,19921,6270,24591
20519,47,TN,189,Wilson County,8,2370,180,2130,50,1700,...,2370,230759,1160,1817,1080,3775,1260,39698,880,9411


In [58]:
TN_income.shape

(768, 148)

In [61]:
my_list = ['STATE', 'COUNTYNAME', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4', 'N2', 'NUMDEP', 'ELDERLY', 'A00100', 'N02650', 'A02650', 'N02300', 'A02300']

In [64]:
TN_income_short = TN_income[my_list]

In [65]:
TN_income_short.columns = ['state', 'county', 'income_bucket', 'return_count', 'single_returns', 'joint_returns', 'head_of_house_returns', 'exemptions', 'dependents', 'elderly', 'agi', 'returns_with_total_inc','total_inc_amt', 'returns_with_unemployment', 'unemployment_comp']